In [15]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime
import urllib


# ========================================
# Función para convertir la URL de Google Sheet en CSV
# ========================================
def get_google_sheet_csv_url(sheet_url: str) -> str:
    file_id = sheet_url.split("/d/")[1].split("/")[0]
    return f"https://docs.google.com/spreadsheets/d/{file_id}/export?format=csv"

# ========================================
# Función para cargar y transformar los datos
# ========================================
def ingest_and_clean_google_sheet(sheet_url: str) -> pd.DataFrame:
    csv_url = get_google_sheet_csv_url(sheet_url)
    df = pd.read_csv(csv_url)
    df.columns = [col.strip().lower().replace(" ", "_") for col in df.columns]
    df["fecha_ingesta"] = datetime.today().strftime('%Y-%m-%d')
    return df

# ========================================
# Función para cargar en PostgreSQL (Docker)
# ========================================
def load_to_postgresql(df: pd.DataFrame, table_name: str, user: str, password: str, host: str, port: str, db: str):
    # Codifica la contraseña si tiene caracteres especiales
    password = urllib.parse.quote_plus(password)

    # Crea la URL de conexión para SQLAlchemy
    engine_url = f"postgresql://{user}:{password}@{host}:{port}/{db}"
    
    # Crear engine SQLAlchemy
    engine = create_engine(engine_url)

    # Carga en la tabla (append = agrega, replace = reemplaza)
    df.to_sql(table_name, engine, if_exists="append", index=False)
    print(f"✅ Datos cargados exitosamente en la tabla: {table_name}")

# ========================================
# PARÁMETROS
# ========================================

# URL de tu Google Sheet (asegurate de que esté compartida públicamente o como "Cualquiera con el enlace")
# sheet_url = "https://docs.google.com/spreadsheets/d/1ukJBMyl7lVsP9XpCgnbU0k-Va9Z10HlX/edit?usp=drive_link&ouid=106134657838899716490&rtpof=true&sd=true"
sheet_url = "https://docs.google.com/spreadsheets/d/1z-KDtW1Rj3bri3gCtE3-G-VKHawCc2HK/edit?usp=drive_link&ouid=106134657838899716490&rtpof=true&sd=true"


# Datos de tu PostgreSQL en Docker
POSTGRES_USER = "postgres"       # Usuario de la BD
POSTGRES_PASSWORD = "123456"  # Contraseña de la BD
POSTGRES_HOST = "localhost"     # Dirección del contenedor (localhost si estás trabajando en tu máquina local)
POSTGRES_PORT = "5431"         # Puerto que mapea el contenedor
POSTGRES_DB = "postgres"      # Nombre de la base de datos
TABLE_NAME = "sales_data"  # Nombre de la tabla a la que cargarás los datos

# ========================================
# FLUJO DE EJECUCIÓN
# ========================================
df_final = ingest_and_clean_google_sheet(sheet_url)
load_to_postgresql(
    df=df_final,
    table_name=TABLE_NAME,
    user=POSTGRES_USER,
    password=POSTGRES_PASSWORD,
    host=POSTGRES_HOST,
    port=POSTGRES_PORT,
    db=POSTGRES_DB
)


✅ Datos cargados exitosamente en la tabla: sales_data


In [18]:
import pandas as pd
from sqlalchemy import create_engine
from datetime import datetime
import urllib

# ========================================
# Función para convertir la URL de Google Sheet en CSV
# ========================================
def get_google_sheet_csv_url(sheet_url: str) -> str:
    # Extraer el ID del archivo de la URL de Google Sheets
    file_id = sheet_url.split("/d/")[1].split("/")[0]
    return f"https://docs.google.com/spreadsheets/d/{file_id}/export?format=csv"

# ========================================
# Función para leer y limpiar el archivo CSV
# ========================================
def ingest_and_clean_google_sheet(sheet_url: str) -> pd.DataFrame:
    csv_url = get_google_sheet_csv_url(sheet_url)
    df = pd.read_csv(csv_url)
    
    # Limpiar las columnas: quitar espacios y pasar a minúsculas
    df.columns = [col.strip().lower().replace(" ", "_") for col in df.columns]
    
    # Agregar una columna de fecha de ingesta (fecha actual)
    df["fecha_ingesta"] = datetime.today().strftime('%Y-%m-%d')
    
    return df

# ========================================
# Función para cargar los datos a PostgreSQL
# ========================================
def load_to_postgresql(df: pd.DataFrame, table_name: str, user: str, password: str, host: str, port: str, db: str):
    # Codificar la contraseña para manejar caracteres especiales
    password = urllib.parse.quote_plus(password)
    
    # Crear la URL de conexión para SQLAlchemy
    engine_url = f"postgresql://{user}:{password}@{host}:{port}/{db}"
    
    # Crear engine de SQLAlchemy
    engine = create_engine(engine_url)
    
    # Cargar los datos a la tabla especificada (usando append para agregar nuevos registros)
    df.to_sql(table_name, engine, if_exists="append", index=False)
    print(f"✅ Datos cargados exitosamente en la tabla: {table_name}")

# ========================================
# Función para cargar múltiples Google Sheets
# ========================================
def load_multiple_sheets(sheet_urls, table_name, user, password, host, port, db):
    for sheet_url in sheet_urls:
        try:
            print(f"Procesando el archivo de Google Sheets: {sheet_url}")
            df = ingest_and_clean_google_sheet(sheet_url)  # Leer y limpiar el archivo
            load_to_postgresql(df, table_name, user, password, host, port, db)  # Cargar los datos
        except Exception as e:
            print(f"❌ Error al procesar el archivo {sheet_url}: {str(e)}")

# ========================================
# Parámetros de conexión a PostgreSQL
# ========================================
POSTGRES_USER = "postgres"         # Usuario de la BD
POSTGRES_PASSWORD = "123456"       # Contraseña de la BD
POSTGRES_HOST = "localhost"        # Dirección del contenedor
POSTGRES_PORT = "5431"            # Puerto mapeado del contenedor
POSTGRES_DB = "postgres"          # Nombre de la base de datos
TABLE_NAME = "sales_data"         # Nombre de la tabla en PostgreSQL

# ========================================
# URLs de Google Sheets a cargar
# ========================================
sheet_urls = [
    "https://docs.google.com/spreadsheets/d/1ukJBMyl7lVsP9XpCgnbU0k-Va9Z10HlX/edit?usp=drive_link&ouid=106134657838899716490&rtpof=true&sd=true"
    "https://docs.google.com/spreadsheets/d/1z-KDtW1Rj3bri3gCtE3-G-VKHawCc2HK/edit?usp=drive_link&ouid=106134657838899716490&rtpof=true&sd=true"


    # Agregar más URLs según sea necesario
]

# ========================================
# Ejecutar carga de múltiples Google Sheets
# ========================================
load_multiple_sheets(sheet_urls, TABLE_NAME, POSTGRES_USER, POSTGRES_PASSWORD, POSTGRES_HOST, POSTGRES_PORT, POSTGRES_DB)


Procesando el archivo de Google Sheets: https://docs.google.com/spreadsheets/d/1ukJBMyl7lVsP9XpCgnbU0k-Va9Z10HlX/edit?usp=drive_link&ouid=106134657838899716490&rtpof=true&sd=truehttps://docs.google.com/spreadsheets/d/1z-KDtW1Rj3bri3gCtE3-G-VKHawCc2HK/edit?usp=drive_link&ouid=106134657838899716490&rtpof=true&sd=true
✅ Datos cargados exitosamente en la tabla: sales_data
